In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
# import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC,NuSVC,LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
print("imported the needed libraries")

imported the needed libraries


In [58]:
data = pd.read_csv("./T_BZL_POL_WithTeamNames.csv")

In [59]:
data=data.drop(['Date','month','year'],axis=1)
home_team=pd.get_dummies(data.home_team, prefix='Home')
away_team=pd.get_dummies(data.away_team, prefix='Away')
data=data.drop(['home_team','away_team'],axis=1)
frames = [data,home_team, away_team]
result = pd.concat(frames,axis=1)
result.to_csv('./T_BZL_POL_withOneHot.csv',index=False)

In [60]:
df = pd.read_csv("./T_BZL_POL_withOneHot.csv")

In [61]:
df.head()

,away_current_pos,away_last_yr_pos,away_prev_game_perf,away_team_av_points,away_team_av_points_conceded,away_team_away_form,away_team_form,away_team_rest_time,away_win_percentage,h2h_form,home_current_pos,home_last_yr_pos,home_prev_game_perf,home_team_av_points,home_team_av_points_conceded,home_team_form,home_team_home_form,home_team_rest_time,home_win_percentage,match_importance,result,Home_AZS Czestochowa,Home_America Volei,Home_Bedzin,Home_Belchatow,Home_Bento Volei,Home_Bielsko-Biala,Home_Blumenau,Home_Brasil Volei Clube,Home_Bydgoszcz,Home_Canoas,Home_Cimed,Home_Corinthians/Guarulhos,Home_Cuprum,Home_Czarni Radom,Home_Fatima UCS,Home_Funvic,Home_GKS Katowice,Home_Gdansk,Home_JF Volei,Home_Jastrzebski,Home_Kedzierzyn-Kozle,Home_Kielce,Home_Londrina,Home_Maringa,Home_Medley,Home_Minas,Home_Olsztyn,Home_Pinheiros,Home_Ponta Grossa,Home_RJX,Home_Rzeszow,Home_SESC-RJ,Home_Sada Cruzeiro,Home_Santo Andre,Home_Sao Bernardo,Home_Sao Jose dos Campos,Home_Sao Judas,Home_Sesi,Home_Slepsk Suwalki,Home_Super Imperatriz,Home_Szczecin,Home_UM Itapetininga,Home_Ulbra Sao Caetano,Home_Verva Warszawa,Home_Volei Futuro,Home_Volei Renata,Home_Volei Ribeirao,Home_Voleisul/Paqueta,Home_Volta Redonda,Home_Wielun,Home_Zawiercie,Away_AZS Czestochowa,Away_America Volei,Away_Bedzin,Away_Belchatow,Away_Bento Volei,Away_Bielsko-Biala,Away_Blumenau,Away_Brasil Volei Clube,Away_Bydgoszcz,Away_Canoas,Away_Cimed,Away_Corinthians/Guarulhos,Away_Cuprum,Away_Czarni Radom,Away_Fatima UCS,Away_Funvic,Away_GKS Katowice,Away_Gdansk,Away_JF Volei,Away_Jastrzebski,Away_Kedzierzyn-Kozle,Away_Kielce,Away_Londrina,Away_Maringa,Away_Medley,Away_Minas,Away_Olsztyn,Away_Pinheiros,Away_Ponta Grossa,Away_RJX,Away_Rzeszow,Away_SESC-RJ,Away_Sada Cruzeiro,Away_Santo Andre,Away_Sao Bernardo,Away_Sao Jose dos Campos,Away_Sao Judas,Away_Sesi,Away_Slepsk Suwalki,Away_Super Imperatriz,Away_Szczecin,Away_UM Itapetininga,Away_Ulbra Sao Caetano,Away_Verva Warszawa,Away_Volei Futuro,Away_Volei Renata,Away_Volei Ribeirao,Away_Voleisul/Paqueta,Away_Volta Redonda,Away_Wielun,Away_Zawiercie
0,12,12,0,20.0,20.00,0.0,0.0,7,50.0,0.0,2,2,0,20.00,20.00,0.0,0.0,7,50.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,6,0,20.0,20.00,0.0,0.0,7,50.0,0.0,8,1,0,20.00,20.00,0.0,0.0,7,50.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,13,3,0,20.0,20.00,0.0,0.0,7,50.0,0.0,11,5,0,20.00,20.00,0.0,0.0,7,50.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15,8,0,20.0,20.00,0.0,0.0,7,50.0,0.0,11,4,0,20.00,20.00,0.0,0.0,7,50.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,13,3,0,20.4,20.64,-0.2,-0.2,2,0.0,0.0,3,4,1,20.85,20.55,0.4,0.4,2,100.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [62]:

print(df.columns)
print(len(df.columns))

Index(['away_current_pos', 'away_last_yr_pos', 'away_prev_game_perf',
       'away_team_av_points', 'away_team_av_points_conceded',
       'away_team_away_form', 'away_team_form', 'away_team_rest_time',
       'away_win_percentage', 'h2h_form',
       ...
       'Away_UM Itapetininga', 'Away_Ulbra Sao Caetano', 'Away_Verva Warszawa',
       'Away_Volei Futuro', 'Away_Volei Renata', 'Away_Volei Ribeirao',
       'Away_Voleisul/Paqueta', 'Away_Volta Redonda', 'Away_Wielun',
       'Away_Zawiercie'],
      dtype='object', length=123)
123


In [63]:
df = df.drop(["away_team_rest_time"], axis=1)
# Due to high correlation with 'away_team_rest_time'

In [64]:
y = df["result"]
X = df.drop(["result"], axis=1)

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state = 121)

In [65]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)


scaled_X_train = pd.DataFrame(data=scaled_X_train, columns=X_train.columns)
scaled_X_test = pd.DataFrame(data=scaled_X_test, columns=X_test.columns)

In [66]:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)


In [67]:
LR = LogisticRegression()

LR.fit(scaled_X_train,y_train)
y_pred = LR.predict(scaled_X_test)
y_train_pred = LR.predict(scaled_X_train)
# accuracy_score(y_test,y_pred)

LR_test_accuracy = accuracy_score(y_test, y_pred)
LR_train_accuracy = accuracy_score(y_train,y_train_pred)
LR_test_f1 = f1_score(y_test,y_pred)
LR_train_f1 = f1_score(y_train,y_train_pred)
LR_test_roc = roc_auc_score(y_test,y_pred)
LR_train_roc = roc_auc_score(y_train,y_train_pred)
LR_test_accuracy

0.7293934681181959

In [68]:
dtree = DecisionTreeClassifier()

dtree.fit(scaled_X_train,y_train)
y_pred = dtree.predict(scaled_X_test)
y_train_pred = dtree.predict(scaled_X_train)
# accuracy_score(y_test,y_pred)

dtree_test_accuracy = accuracy_score(y_test, y_pred)
dtree_train_accuracy = accuracy_score(y_train,y_train_pred)
dtree_test_f1 = f1_score(y_test,y_pred)
dtree_train_f1 = f1_score(y_train,y_train_pred)
dtree_test_roc = roc_auc_score(y_test,y_pred)
dtree_train_roc = roc_auc_score(y_train,y_train_pred)
dtree_test_accuracy

0.6376360808709176

In [69]:
SVM = SVC(probability = True)

SVM.fit(scaled_X_train,y_train)
y_pred = SVM.predict(scaled_X_test)
y_train_pred = SVM.predict(scaled_X_train)
# accuracy_score(y_test,y_pred)

SVM_test_accuracy = accuracy_score(y_test, y_pred)
SVM_train_accuracy = accuracy_score(y_train,y_train_pred)
SVM_test_f1 = f1_score(y_test,y_pred)
SVM_train_f1 = f1_score(y_train,y_train_pred)
SVM_test_roc = roc_auc_score(y_test,y_pred)
SVM_train_roc = roc_auc_score(y_train,y_train_pred)
SVM_test_accuracy

0.6998444790046656

In [70]:
RForest = RandomForestClassifier()

RForest.fit(scaled_X_train,y_train)
y_pred = RForest.predict(scaled_X_test)
y_train_pred = RForest.predict(scaled_X_train)
# accuracy_score(y_test,y_pred)

RForest_test_accuracy = accuracy_score(y_test, y_pred)
RForest_train_accuracy = accuracy_score(y_train,y_train_pred)
RForest_test_f1 = f1_score(y_test,y_pred)
RForest_train_f1 = f1_score(y_train,y_train_pred)
RForest_test_roc = roc_auc_score(y_test,y_pred)
RForest_train_roc = roc_auc_score(y_train,y_train_pred)
RForest_test_accuracy

0.6998444790046656

In [71]:
KNN = KNeighborsClassifier()

KNN.fit(scaled_X_train,y_train)
y_pred = KNN.predict(scaled_X_test)
y_train_pred = KNN.predict(scaled_X_train)
# accuracy_score(y_test,y_pred)

KNN_test_accuracy = accuracy_score(y_test, y_pred)
KNN_train_accuracy = accuracy_score(y_train,y_train_pred)
KNN_test_f1 = f1_score(y_test,y_pred)
KNN_train_f1 = f1_score(y_train,y_train_pred)
KNN_test_roc = roc_auc_score(y_test,y_pred)
KNN_train_roc = roc_auc_score(y_train,y_train_pred)
KNN_test_accuracy



0.6438569206842923

In [72]:
bayes = GaussianNB()

bayes.fit(scaled_X_train,y_train)
y_pred = bayes.predict(scaled_X_test)
y_train_pred = bayes.predict(scaled_X_train)
# accuracy_score(y_test,y_pred)

bayes_test_accuracy = accuracy_score(y_test, y_pred)
bayes_train_accuracy = accuracy_score(y_train,y_train_pred)
bayes_test_f1 = f1_score(y_test,y_pred)
bayes_train_f1 = f1_score(y_train,y_train_pred)
bayes_test_roc = roc_auc_score(y_test,y_pred)
bayes_train_roc = roc_auc_score(y_train,y_train_pred)
bayes_test_accuracy


0.6780715396578538

In [73]:
LDA = LinearDiscriminantAnalysis(solver='lsqr')

LDA.fit(scaled_X_train,y_train)
y_pred = LDA.predict(scaled_X_test)
y_train_pred = LDA.predict(scaled_X_train)
# accuracy_score(y_test,y_pred)

LDA_test_accuracy = accuracy_score(y_test, y_pred)
LDA_train_accuracy = accuracy_score(y_train,y_train_pred)
LDA_test_f1 = f1_score(y_test,y_pred)
LDA_train_f1 = f1_score(y_train,y_train_pred)
LDA_test_roc = roc_auc_score(y_test,y_pred)
LDA_train_roc = roc_auc_score(y_train,y_train_pred)
LDA_test_accuracy



0.7356143079315708

In [74]:
QDA = QuadraticDiscriminantAnalysis()

QDA.fit(scaled_X_train,y_train)
y_pred = QDA.predict(scaled_X_test)
y_train_pred = QDA.predict(scaled_X_train)
# accuracy_score(y_test,y_pred)

QDA_test_accuracy = accuracy_score(y_test, y_pred)
QDA_train_accuracy = accuracy_score(y_train,y_train_pred)
QDA_test_f1 = f1_score(y_test,y_pred)
QDA_train_f1 = f1_score(y_train,y_train_pred)
QDA_test_roc = roc_auc_score(y_test,y_pred)
QDA_train_roc = roc_auc_score(y_train,y_train_pred)
QDA_test_accuracy


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.6205287713841369

In [79]:
from sklearn.neural_network import MLPClassifier

# scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
# clf = make_pipeline(preprocessing.StandardScaler(), MLPClassifier(solver='adam', alpha=3e-4, hidden_layer_sizes=(10), random_state=1,max_iter=1000))
# clf2 = make_pipeline(preprocessing.MinMaxScaler(), MLPClassifier(solver='adam', alpha=3e-4, hidden_layer_sizes=(10), random_state=1,max_iter=1000))

# scores = cross_validate(clf, X, y, scoring=scoring, cv=cv, return_train_score = True)
# scores2 = cross_validate(clf2, X, y, scoring=scoring, cv=cv, return_train_score = True)

MLP = MLPClassifier(solver='adam',learning_rate = 'adaptive',hidden_layer_sizes=(5), random_state=202,max_iter=500,activation = 'tanh')

MLP.fit(scaled_X_train,y_train)
y_pred = MLP.predict(scaled_X_test)
y_train_pred = MLP.predict(scaled_X_train)
# accuracy_score(y_test,y_pred)

MLP_test_accuracy = accuracy_score(y_test, y_pred)
MLP_train_accuracy = accuracy_score(y_train,y_train_pred)
MLP_test_f1 = f1_score(y_test,y_pred)
MLP_train_f1 = f1_score(y_train,y_train_pred)
MLP_test_roc = roc_auc_score(y_test,y_pred)
MLP_train_roc = roc_auc_score(y_train,y_train_pred)
MLP_test_accuracy



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.6531881804043546

In [80]:
models_initial = pd.DataFrame({
    'Model'         : ['Logistic Regression', 'Decision Tree', 'Support Vector Machine', 'Random Forest', 'K-Nearest Neighbors', 'Naive Bayes', 'LinearDiscriminantAnalysis', 'QuadraticDiscriminantAnalysis','Neural Network'],
    'test_Acc'       : [LR_test_accuracy, dtree_test_accuracy, SVM_test_accuracy, RForest_test_accuracy, KNN_test_accuracy, bayes_test_accuracy, LDA_test_accuracy, QDA_test_accuracy,MLP_test_accuracy],
    'train_Accuracy'       : [LR_train_accuracy, dtree_train_accuracy, SVM_train_accuracy, RForest_train_accuracy, KNN_train_accuracy, bayes_train_accuracy, LDA_train_accuracy, QDA_train_accuracy,MLP_train_accuracy],
    'test_AUC_ROC'   : [LR_test_roc, dtree_test_roc, SVM_test_roc, RForest_test_roc, KNN_test_roc, bayes_test_roc, LDA_test_roc, QDA_test_roc,MLP_test_roc],
    'train_AUC_ROC'        : [LR_train_roc, dtree_train_roc, SVM_train_roc, RForest_train_roc, KNN_train_roc, bayes_train_roc, LDA_train_roc, QDA_train_roc,MLP_train_roc],
    'train_F1_Score'       : [LR_train_f1, dtree_train_f1, SVM_train_f1, RForest_train_f1, KNN_train_f1, bayes_train_f1, LDA_train_f1, QDA_train_f1,MLP_train_f1],
    'test_F1_Score'  : [LR_test_f1, dtree_test_f1, SVM_test_f1, RForest_test_f1, KNN_test_f1, bayes_test_f1, LDA_test_f1, QDA_test_f1,MLP_test_f1]
    
    }, columns = ['Model', 'test_Acc', 'train_Accuracy', 'test_AUC_ROC', 'train_AUC_ROC','test_F1_Score', 'train_F1_Score'])
print("StandardScaler preprocessing")
models_sorted=models_initial.sort_values(by='test_Acc', ascending=False)
pd.set_option("display.max_rows",None, "display.max_columns", None)
print(models_sorted)



StandardScaler preprocessing
                           Model  test_Acc  train_Accuracy  test_AUC_ROC  \
6     LinearDiscriminantAnalysis  0.735614        0.755832      0.732128   
0            Logistic Regression  0.729393        0.759720      0.725879   
2         Support Vector Machine  0.699844        0.820373      0.694840   
3                  Random Forest  0.699844        1.000000      0.696093   
5                    Naive Bayes  0.678072        0.723173      0.669835   
8                 Neural Network  0.653188        0.876361      0.650272   
4            K-Nearest Neighbors  0.643857        0.781104      0.640063   
1                  Decision Tree  0.637636        1.000000      0.636740   
7  QuadraticDiscriminantAnalysis  0.620529        0.673795      0.602318   

   train_AUC_ROC  test_F1_Score  train_F1_Score  
6       0.749761       0.759887        0.785226  
0       0.754448       0.754237        0.787337  
2       0.815714       0.732316        0.841128  
3       1.